In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio

from rasterio.warp import calculate_default_transform, reproject, Resampling

matplotlib.rc('figure', dpi=125)

In [2]:
# proj crs NAD 1983 UTM Zone 18N
# geographic crs NAD 1983

def couple(surge_fpath, riv_fpath, compound_fpath):

    with rio.open(surge_fpath) as ds_in:
        surge_inun = ds_in.read(1, masked=True)
        surge_inun_profile = ds_in.profile
    msk = surge_inun.copy().mask # save mask, will get wiped out on next lines
    surge_inun[surge_inun>3.4e+38] = np.nan # set nodata
    surge_inun[surge_inun < 0] = 0
    surge_inun = np.ma.masked_array(surge_inun, msk)
    # save mask with GDAL convetion for writing later
    # https://rasterio.readthedocs.io/en/latest/topics/masks.html
    w_msk = (~surge_inun.mask * 255).astype('uint8')

    with rio.open(riv_fpath) as ds_in:
        riv_inun = ds_in.read(1, masked=True)
    msk = riv_inun.copy().mask # save mask, will get wiped out on next lines
    riv_inun[riv_inun>3.4e+38] = np.nan # set nodata
    riv_inun[riv_inun<0] = 0
    riv_inun = np.ma.masked_array(riv_inun, msk)

    # give riv_inun one extra row and col to match shapes with surge_inun
    newrow = np.empty(riv_inun.shape[1])
    newrow.fill(np.nan)
    riv_inun = np.vstack([riv_inun,newrow])

    newcol = np.empty([riv_inun.shape[0],1])
    newcol.fill(np.nan)
    riv_inun = np.hstack([riv_inun,newcol])

    # avoid divide by zero errors
    ratio_adj = 0.001 # 0.001 m = 0.04 in
    ratio_array = (surge_inun + ratio_adj) / (riv_inun + ratio_adj)

    # calculate compound inundation accounting for transition zone
    compound_inun = np.where(ratio_array>1,surge_inun,
                        np.where(ratio_array<0.5,riv_inun,
                            ((np.maximum(surge_inun,riv_inun) + (surge_inun+riv_inun))/2)
                                ))

    # write data
    with rio.open(compound_fpath, 'w', **surge_inun_profile) as ds_out:
        ds_out.write(compound_inun,1)
        ds_out.write_mask(w_msk)

    return compound_inun

In [3]:
# make compound inundation maps corresponding to satellite image timestamps

_ = couple(
    'data_github/inun_surge_201809020600.tif',
    'data_github/030202_201809020600_depth_map_projUTM18n.tif',
    'data_github/compound_inun_201809020600.tif'
)

_ = couple(
    'data_github/florence_surge_nhc_max.tif',
    'data_github/030202_201809140600_depth_map_projUTM18n.tif',
    'data_github/compound_inun_201809140600.tif'
)

_ = couple(
    'data_github/inun_surge_201809192100.tif',
    'data_github/030202_201809192100_depth_map_projUTM18n.tif',
    'data_github/compound_inun_201809192100.tif'
)